In [1]:
### Notebook that introduces the decorators in Python, and its use-cases

In [2]:
### A decorator takes another function & extends the behavior of the latter function without modifying it

In [3]:
### A function returns a value based on the argument
def add_one(number):
    return number + 1
print(add_one(2))

3


In [4]:
### function can be passed as arguments
def say_hello(name):
    return f"Hi there {name}"

def be_awesome(name):
    return f"lets make things super fast... yo {name}"

def greet_bob(greeter_func):
    # Function is written for doing a particular activity for particular object/ instance
    return greeter_func("Bob")

In [5]:
greet_bob(be_awesome)

'lets make things super fast... yo Bob'

In [6]:
greet_bob(say_hello)

'Hi there Bob'

In [7]:
def parent():
    print("Parent function is printing...")

    def first_child():
        print("Printing from the first child function")
    
    def second_child():
        print("Printing from the second child function")
    
    first_child()
    second_child()

In [8]:
parent()

Parent function is printing...
Printing from the first child function
Printing from the second child function


In [10]:
# function with () means the function is evaluated, while without () means the reference of the function

def parent(num):
    def first_child():
        return "hi.. child 1 here"

    def second():
        return "No 2 reporting"

    if num == 1:
        return first_child

    else:
        return second
    
one = parent(1)  # the function is returned and assigned to variable one & two
two = parent(2)

In [11]:
two()

'No 2 reporting'

In [13]:
def decorator(func):
    """Function that takes another function as argument, and does some processing before and after
    calling it. It has internal function"""
    # declare a wrapper function
    def wrapper():
        # do before op
        print("operation before")
        func()  # call function
        # do after op
        print("operation after")
    # return wrapper 
    return wrapper

def say_whee():
    # print something
    print('whee')
    # return None

say_whee = decorator(say_whee)
say_whee

<function __main__.decorator.<locals>.wrapper()>

In [14]:
say_whee()

operation before
whee
operation after


In [23]:
from datetime import datetime

def not_during_the_night(func):
    def wrapper():
        #  if the time is between 7 to 10, then execute function
        if 7 <= datetime.now().hour < 22:
            func()
        else:
            pass
    return wrapper

def say_whee():
    print("whee!")

say_whee = not_during_the_night(say_whee)

In [19]:
say_whee()

In [24]:
@not_during_the_night
def say_whee():
    print('Nice')

x = say_whee()
x

Nice


In [27]:
def twice_do(func):
    def wrapper_do():
        func()
        func()
    return wrapper_do

@twice_do
def say_hello():
    print(f"this is decorated")

say_hello()

this is decorated
this is decorated


In [28]:
say_hello('hero here')

## The solution is to use *args and **kwargs in the inner wrapper function. Then it will accept an arbitrary number of positional and keyword arguments. 

TypeError: twice_do.<locals>.wrapper_do() takes 0 positional arguments but 1 was given

In [29]:
def do_twice_args(func):
    def twice_args(*args, **kwargs):
        func(*args, **kwargs)
        func(*args, **kwargs)
    return twice_args

In [31]:
@do_twice_args
def say_args(name):
    print(f"The name is: {name}")

say_args('newYork')

The name is: newYork
The name is: newYork


### Oops, your decorator ate the return value from the function.

Introspection is the ability of an object to know about its own attributes at runtime.

In [3]:
# Outer decorator function, that takes a function
def once_return(func):
    # No operation on func, another process function created
    def process(*args, **kwargs):
        # call the func with *args
        func(*args)
        # can return like regular function
        return func(*args)
    # return the interal process function
    return process

In [4]:
@once_return
def say_ret(name):
    print(f'my name is: {name}')
    return name

say_ret('Corsica')

my name is: Corsica
my name is: Corsica


'Corsica'

In [1]:
print

<function print(*args, sep=' ', end='\n', file=None, flush=False)>

In [6]:
print(say_ret.__name__)

help(say_ret)

process
Help on function process in module __main__:

process(*args, **kwargs)
    # No operation on func, another process function created



In [8]:
import functools

def do_twice(func):
    @functools.wraps(func)
    def wrapper_do_twice(*args, **kwargs):
        func(*args, **kwargs)
        return func(*args, **kwargs)

    return wrapper_do_twice

In [9]:
import functools
import time

def timer(func):
    """Print the runtime of the decorated function"""
    @functools.wraps(func)
    def wrapper_timer(*args, **kwargs):
        start_time = time.perf_counter()    # 1
        value = func(*args, **kwargs)
        end_time = time.perf_counter()      # 2
        run_time = end_time - start_time    # 3
        print(f"Finished {func.__name__!r} in {run_time:.4f} secs")
        return value
    return wrapper_timer

In [11]:
@timer
def waste_some_time(num_times):
    for _ in range(num_times):
        sum([i**2 for i in range(10000)])

waste_some_time(100)

Finished 'waste_some_time' in 0.2306 secs


In [12]:
import functools

def debug(func):
    """Print the function signature and return value"""
    @functools.wraps(func)
    def wrapper_debug(*args, **kwargs):
        args_repr = [repr(a) for a in args]                      # 1
        kwargs_repr = [f"{k}={v!r}" for k, v in kwargs.items()]  # 2
        signature = ", ".join(args_repr + kwargs_repr)           # 3
        print(f"Calling {func.__name__}({signature})")
        value = func(*args, **kwargs)
        print(f"{func.__name__!r} returned {value!r}")           # 4
        return value
    return wrapper_debug

In [13]:
@debug
def make_greeting(name, age=None):
    if age is None:
        return f"Howdy {name}!"
    else:
        return f"Whoa {name}! {age} already, you are growing up!"

In [14]:
make_greeting("Benjamin")

Calling make_greeting('Benjamin')
'make_greeting' returned 'Howdy Benjamin!'


'Howdy Benjamin!'

In [20]:
import math

math.fact = debug(math.factorial)

def approx_e(terms = 10):
    return sum(1 / math.fact(n) for n in range(terms))

In [21]:
approx_e(5)

Calling factorial(0)
'factorial' returned 1
Calling factorial(1)
'factorial' returned 1
Calling factorial(2)
'factorial' returned 2
Calling factorial(3)
'factorial' returned 6
Calling factorial(4)
'factorial' returned 24


2.708333333333333

In [22]:
def slow_down(func):
    """Sleep 1 second before calling the function"""
    @functools.wraps(func)
    def wrapper_slow_down(*args, **kwargs):
        time.sleep(1)
        return func(*args, **kwargs)
    return wrapper_slow_down

In [23]:
@slow_down
def countdown(from_number):
    if from_number < 1:
        print("Liftoff!")
    else:
        print(from_number)
        countdown(from_number - 1)

In [24]:
countdown(2)

2
1
Liftoff!


In [25]:
globals()

{'__name__': '__main__',
 '__doc__': 'Automatically created module for IPython interactive environment',
 '__package__': None,
 '__loader__': None,
 '__spec__': None,
 '__builtin__': <module 'builtins' (built-in)>,
 '__builtins__': <module 'builtins' (built-in)>,
 '_ih': ['',
  'print',
  'say_ret.__name__',
  '# Outer decorator function, that takes a function\ndef once_return(func):\n    # No operation on func, another process function created\n    def process(*args, **kwargs):\n        # call the func with *args\n        func(*args)\n        # can return like regular function\n        return func(*args)\n    # return the interal process function\n    return process',
  "@once_return\ndef say_ret(name):\n    print(f'my name is: {name}')\n    return name\n\nsay_ret('Corsica')",
  'say_ret.__name__',
  'print(say_ret.__name__)\n\nhelp(say_ret)',
  'import functools\n\ndef do_twice(func):\n    @functools.wraps(func)\n    def wrapper_do_twice(*args, **kwargs):\n        func(*args, **kwarg

In [26]:
# Here the methods of the class is decorated
class TimeWaster:
    @debug
    def __init__(self, max_num):
        self.max_num = max_num

    @timer
    def waste_time(self, num_times):
        for _ in range(num_times):
            sum([i**2 for i in range(self.max_num)])

In [27]:
tw = TimeWaster(1000)

tw.waste_time(999)

Calling __init__(<__main__.TimeWaster object at 0x0000020A37BB8110>, 1000)
'__init__' returned None
Finished 'waste_time' in 0.3565 secs


A common use of class decorators is to be a simpler alternative to some use-cases of metaclasses. In both cases, you are changing the definition of a class dynamically.

In [29]:
class TimeWaster:
    @debug
    def __init__(self, max_num):
        self.max_num = max_num

    @timer
    def waste_time(self, num_times):
        for _ in range(num_times):
            sum([i**2 for i in range(self.max_num)])

In [30]:
@debug
@do_twice
def greet(name):
    print(f"Hello {name}")

In [31]:
greet('eve')

Calling greet('eve')
Hello eve
Hello eve
'greet' returned None


In [34]:
def repeat(num_times):
    def decorator_repeat(func):
        @functools.wraps(func)
        def wrapper_repeat(*args, **kwargs):
            for _ in range(num_times):
                value = func(*args, **kwargs)
            return value
        return wrapper_repeat
    return decorator_repeat

In [37]:
def repeat(_func=None, *, num_times=2):
    def decorator_repeat(func):
        @functools.wraps(func)
        def wrapper_repeat(*args, **kwargs):
            for _ in range(num_times):
                value = func(*args, **kwargs)
            return value
        return wrapper_repeat

    if _func is None:
        return decorator_repeat
    else:
        return decorator_repeat(_func)

In [38]:
@repeat(num_times=4)
def greet(name):
    print(f"Hello {name}")

In [39]:
greet("euro")

Hello euro
Hello euro
Hello euro
Hello euro


In [36]:
def wrapper_repeat(*args, **kwargs):
    for _ in range(num_times):
        value = func(*args, **kwargs)
    return value

The num_times argument is seemingly not used in repeat() itself. But by passing num_times a closure is created where the value of num_times is stored until it will be used later by wrapper_repeat().

In [40]:
import functools

def count_calls(func):
    @functools.wraps(func)
    def wrapper_count_calls(*args, **kwargs):
        wrapper_count_calls.num_calls += 1
        print(f"Call {wrapper_count_calls.num_calls} of {func.__name__!r}")
        return func(*args, **kwargs)
    wrapper_count_calls.num_calls = 0
    return wrapper_count_calls

In [41]:
# For a class instance to be callable, you implement the special .__call__() method:

class Counter:
    def __init__(self, start=0):
        self.count = start

    def __call__(self):
        self.count += 1
        print(f"Current count is {self.count}")

In [42]:
import functools

class CountCalls:
    def __init__(self, func):
        functools.update_wrapper(self, func)
        self.func = func
        self.num_calls = 0

    def __call__(self, *args, **kwargs):
        self.num_calls += 1
        print(f"Call {self.num_calls} of {self.func.__name__!r}")
        return self.func(*args, **kwargs)

@CountCalls
def say_whee():
    print("Whee!")

In [44]:
# Using is returns True only for objects that are the exact same instance

@count_calls
def fibonacci(num):
    if num < 2:
        return num
    return fibonacci(num - 1) + fibonacci(num - 2)

fibonacci(10)

Call 1 of 'fibonacci'
Call 2 of 'fibonacci'
Call 3 of 'fibonacci'
Call 4 of 'fibonacci'
Call 5 of 'fibonacci'
Call 6 of 'fibonacci'
Call 7 of 'fibonacci'
Call 8 of 'fibonacci'
Call 9 of 'fibonacci'
Call 10 of 'fibonacci'
Call 11 of 'fibonacci'
Call 12 of 'fibonacci'
Call 13 of 'fibonacci'
Call 14 of 'fibonacci'
Call 15 of 'fibonacci'
Call 16 of 'fibonacci'
Call 17 of 'fibonacci'
Call 18 of 'fibonacci'
Call 19 of 'fibonacci'
Call 20 of 'fibonacci'
Call 21 of 'fibonacci'
Call 22 of 'fibonacci'
Call 23 of 'fibonacci'
Call 24 of 'fibonacci'
Call 25 of 'fibonacci'
Call 26 of 'fibonacci'
Call 27 of 'fibonacci'
Call 28 of 'fibonacci'
Call 29 of 'fibonacci'
Call 30 of 'fibonacci'
Call 31 of 'fibonacci'
Call 32 of 'fibonacci'
Call 33 of 'fibonacci'
Call 34 of 'fibonacci'
Call 35 of 'fibonacci'
Call 36 of 'fibonacci'
Call 37 of 'fibonacci'
Call 38 of 'fibonacci'
Call 39 of 'fibonacci'
Call 40 of 'fibonacci'
Call 41 of 'fibonacci'
Call 42 of 'fibonacci'
Call 43 of 'fibonacci'
Call 44 of 'fibonacc

55

In [45]:
fibonacci.num_calls

177

In [46]:
def cache(func):
    """Keep a cache of previous function calls"""
    @functools.wraps(func)
    def wrapper_cache(*args, **kwargs):
        cache_key = args + tuple(kwargs.items())
        if cache_key not in wrapper_cache.cache:
            wrapper_cache.cache[cache_key] = func(*args, **kwargs)
        return wrapper_cache.cache[cache_key]
    wrapper_cache.cache = dict()
    return wrapper_cache

In [47]:
@cache
@count_calls
def fibonacci(num):
    if num < 2:
        return num
    return fibonacci(num - 1) + fibonacci(num - 2)

In [48]:
import functools

@functools.lru_cache(maxsize=4)
def fibonacci(num):
    print(f"Calculating fibonacci({num})")
    if num < 2:
        return num
    return fibonacci(num - 1) + fibonacci(num - 2)

In [50]:
def set_unit(unit):
    """Register a unit on a function"""
    def decorator_set_unit(func):
        func.unit = unit
        return func
    return decorator_set_unit

In [51]:
import math

@set_unit("cm^3")
def volume(radius, height):
    return math.pi * radius**2 * height

In [52]:
import math

def volume(radius, height) -> "cm^3":
    return math.pi * radius**2 * height

In [53]:
!pip install pint

   ---------------------------------------- 0.0/294.0 kB ? eta -:--:--
   - -------------------------------------- 10.2/294.0 kB ? eta -:--:--
   ---- ---------------------------------- 30.7/294.0 kB 330.3 kB/s eta 0:00:01
   ----- --------------------------------- 41.0/294.0 kB 245.8 kB/s eta 0:00:02
   -------- ------------------------------ 61.4/294.0 kB 328.2 kB/s eta 0:00:01
   -------- ------------------------------ 61.4/294.0 kB 328.2 kB/s eta 0:00:01
   --------- ----------------------------- 71.7/294.0 kB 245.8 kB/s eta 0:00:01
   -------------- ----------------------- 112.6/294.0 kB 312.2 kB/s eta 0:00:01
   ------------------ ------------------- 143.4/294.0 kB 370.8 kB/s eta 0:00:01
   ------------------ ------------------- 143.4/294.0 kB 370.8 kB/s eta 0:00:01
   ------------------- ------------------ 153.6/294.0 kB 316.5 kB/s eta 0:00:01
   ------------------------- ------------ 194.6/294.0 kB 347.1 kB/s eta 0:00:01
   -------------------------- ----------- 204.8/294.0 kB 

ERROR: Could not install packages due to an OSError: [WinError 2] The system cannot find the file specified: 'C:\\Python312\\Scripts\\pint-convert.exe' -> 'C:\\Python312\\Scripts\\pint-convert.exe.deleteme'



In [ ]:
import pint
ureg = pint.UnitRegistry()
vol = volume(3, 5) * ureg(volume.unit)

vol.to("cubic inches")

vol.to("gallons").m  # Magnitude

In [ ]:
def use_unit(unit):
    """Have a function return a Quantity with given unit"""
    use_unit.ureg = pint.UnitRegistry()
    def decorator_use_unit(func):
        @functools.wraps(func)
        def wrapper_use_unit(*args, **kwargs):
            value = func(*args, **kwargs)
            return value * use_unit.ureg(unit)
        return wrapper_use_unit
    return decorator_use_unit

@use_unit("meters per second")
def average_speed(distance, duration):
    return distance / duration

In [ ]:
bolt = average_speed(100, 9.58)

bolt.to("km per hour")

bolt.to("mph").m  # Magnitude

In [54]:
from flask import Flask, request, abort
import functools
app = Flask(__name__)

def validate_json(*expected_args):                  # 1
    def decorator_validate_json(func):
        @functools.wraps(func)
        def wrapper_validate_json(*args, **kwargs):
            json_object = request.get_json()
            for expected_arg in expected_args:      # 2
                if expected_arg not in json_object:
                    abort(400)
            return func(*args, **kwargs)
        return wrapper_validate_json
    return decorator_validate_json

ModuleNotFoundError: No module named 'flask'